In [56]:
import sklearn
import pandas as pd
import numpy as np
from sklearn.model_selection import train_test_split
#1 read in csv file using pandas 

df = pd.read_csv('federalist.csv')
df.dropna(inplace = True)
df = df.astype({'author':'category'})
print(df[:5])
df['author'].value_counts()

#2
train, test = train_test_split(df, test_size=0.2, random_state=1234)
print(test.shape)
print(train.shape)

     author                                               text
0  HAMILTON  FEDERALIST. No. 1 General Introduction For the...
1       JAY  FEDERALIST No. 2 Concerning Dangers from Forei...
2       JAY  FEDERALIST No. 3 The Same Subject Continued (C...
3       JAY  FEDERALIST No. 4 The Same Subject Continued (C...
4       JAY  FEDERALIST No. 5 The Same Subject Continued (C...
(17, 2)
(66, 2)


In [69]:
#3 //proof of concept,testing on one row
from sklearn.feature_extraction.text import TfidfVectorizer
from sklearn.feature_extraction.text import ENGLISH_STOP_WORDS
from nltk.corpus import stopwords
from nltk.tokenize import word_tokenize
textRows = train.text
rowTokens = word_tokenize(textRows.iloc[1])
print(rowTokens[:20])
row_without = [word for word in rowTokens if not word in ENGLISH_STOP_WORDS]
print(row_without[:20])

textTestRows = test.text
rowTestTokens = word_tokenize(textTestRows.iloc[1])
print(rowTestTokens[:20])
testRow_without = [w for w in rowTestTokens if not w in ENGLISH_STOP_WORDS]
print(testRow_without[:20])

tfidf_train = TfidfVectorizer().fit(textRows)
tfidf_test = TfidfVectorizer()
X_train_tf = tfidf_train.transform(textRows)
X_test_tf = tfidf_train.transform(textTestRows)
print(X_train_tf[:7])
print(X_train_tf.shape)
print(X_test_tf.shape)

#tfidf_matrix = tfidf_vectorizer.fit_transform(train)

['FEDERALIST', 'No', '.', '55', 'The', 'Total', 'Number', 'of', 'the', 'House', 'of', 'Representatives', 'From', 'the', 'New', 'York', 'Packet', '.', 'Friday', ',']
['FEDERALIST', 'No', '.', '55', 'The', 'Total', 'Number', 'House', 'Representatives', 'From', 'New', 'York', 'Packet', '.', 'Friday', ',', 'February', '15', ',', '1788']
['FEDERALIST', 'No', '.', '52', 'The', 'House', 'of', 'Representatives', 'From', 'the', 'New', 'York', 'Packet', '.', 'Friday', ',', 'February', '8', ',', '1788']
['FEDERALIST', 'No', '.', '52', 'The', 'House', 'Representatives', 'From', 'New', 'York', 'Packet', '.', 'Friday', ',', 'February', '8', ',', '1788', '.', 'To']
  (0, 7988)	0.010853968071599773
  (0, 7980)	0.00733235756964619
  (0, 7966)	0.01130712513773646
  (0, 7965)	0.012347955814124224
  (0, 7956)	0.018089946785999622
  (0, 7948)	0.007815472727013799
  (0, 7934)	0.004332033689528359
  (0, 7933)	0.005222540515575079
  (0, 7931)	0.014855754936363237
  (0, 7929)	0.04341587228639909
  (0, 7913)	0.

In [71]:
#3  //this preprocesses each row of the text training data removing stopwords and for test data (just takes v long)
from sklearn.feature_extraction.text import TfidfVectorizer
from sklearn.feature_extraction.text import ENGLISH_STOP_WORDS
from nltk.tokenize import word_tokenize
for index, row in train.text.iteritems():
   rowTokens = word_tokenize(row)
   row_without_sw = [word for word in rowTokens if not word in ENGLISH_STOP_WORDS]

for ind, r in test.text.iteritems():
   textTestRows = test.text
   rowTestTokens = word_tokenize(r)
   testRow_without = [w for w in rowTestTokens if not w in ENGLISH_STOP_WORDS]

text_Train = train.text
text_Test = test.text

tfidf_train = TfidfVectorizer().fit(text_Train)
tfidf_test = TfidfVectorizer()
X_train_tf = tfidf_train.transform(text_Train)
X_test_tf = tfidf_train.transform(text_Test)
print(X_train_tf.shape)
print(X_test_tf.shape)

(66, 7996)
(17, 7996)


In [72]:
#4
from sklearn.naive_bayes import BernoulliNB
from sklearn.feature_extraction.text import TfidfVectorizer

clf = BernoulliNB()
clf.fit(X_train_tf, train.author)
score = clf.score(X_train_tf, train.author)
print(score)
clf.fit(X_test_tf, test.author)
scoreTest = clf.score(X_test_tf, test.author)
print(scoreTest)

0.7878787878787878
0.5882352941176471


In [77]:
#vectorization with max_features option set to use only the 1000 most frequent words. In addition to
#the words, add bigrams as a feature. Try Naïve Bayes again on the new train/test vectors and
#compare your results.
#5 
from sklearn.feature_extraction.text import TfidfVectorizer
from sklearn.naive_bayes import BernoulliNB


tfidf_train = TfidfVectorizer(max_features=1000, ngram_range = (2,2)).fit(text_Train)
tfidf_test = TfidfVectorizer(max_features=1000, ngram_range = (2,2))
X_train_tf = tfidf_train.transform(text_Train)
X_test_tf = tfidf_train.transform(text_Test)

clf = BernoulliNB()
clf.fit(X_train_tf, train.author)
score = clf.score(X_train_tf, train.author)
print(score)
clf.fit(X_test_tf, test.author)
scoreTest = clf.score(X_test_tf, test.author)
print(scoreTest)


1.0
1.0


My initial score on the training and test was 78% and 58% and when I set the max features to 100 and added bigrams as a feature, the score was 100% for both. Setting the max features and adding bigrams made the Bernoulli Naive Bayes model fit way better. 

In [92]:
from sklearn.linear_model import LogisticRegression

# all parameters not specified are set to their defaults
logisticRegr = LogisticRegression(C = 15)
logisticRegr.fit(X_train_tf, train.author)
predictions = logisticRegr.predict(X_test_tf)

# Use score method to get accuracy of model
score = logisticRegr.score(X_test_tf, test.author)
print(score)

0.7058823529411765


I adjusted the C parameter for the Logistirc Regression method, from the default of 1 to 15, and that caused my score to go up in accuracy. The C parameter controls the regularization strength. Smaller values = strong regularization, by weakening the regularization on the data, the accuracy increased.

In [121]:
from sklearn.neural_network import MLPClassifier

nn = MLPClassifier(max_iter=400, hidden_layer_sizes=(5,)).fit(X_train_tf, train.author)
scoreT = nn.score(X_test_tf, test.author)
print("first try:", scoreT)

first try: 0.5882352941176471


c:\ProgramData\Anaconda3\lib\site-packages\sklearn\neural_network\_multilayer_perceptron.py:692: ConvergenceWarning: Stochastic Optimizer: Maximum iterations (400) reached and the optimization hasn't converged yet.
  warnings.warn(


In [116]:
nn = MLPClassifier(max_iter=400, hidden_layer_sizes=(100,)).fit(X_train_tf, train.author)
scoreT = nn.score(X_test_tf, test.author)
print("second try:", scoreT)

second try: 0.7058823529411765


In [120]:
nn = MLPClassifier(max_iter=400, hidden_layer_sizes=(7,)).fit(X_train_tf, train.author)
scoreT = nn.score(X_test_tf, test.author)
print("third try:", scoreT)

third try: 0.8235294117647058


c:\ProgramData\Anaconda3\lib\site-packages\sklearn\neural_network\_multilayer_perceptron.py:692: ConvergenceWarning: Stochastic Optimizer: Maximum iterations (400) reached and the optimization hasn't converged yet.
  warnings.warn(


I adjusted the hidden layers and even the max iterations. My final results when my hidden_layers_sizes being changed to (7,), the highest I got the accuracy was 82%